In [ ]:
#export
from sklearn.base import BaseEstimator
import numpy as np
from typing import Union, Callable, Optional, Tuple, List, Iterator, Any
from copy import deepcopy
from dataclasses import dataclass, field

In [ ]:
#default_exp base

In [ ]:
data_types = Union[np.array, None]

In [ ]:
#export
class FitTask(type):
    # https://stackoverflow.com/a/18859019
    def __init__(cls, name, bases, clsdict):
        if 'fit' in clsdict:
            def new_fit(self, **kwargs):
                try:
                    X_training = kwargs["X_training"] if "X_training" in kwargs else self.dataset["X_training"]
                    y_training = kwargs["y_training"] if "y_training" in kwargs else self.dataset["y_training"]
                    t_training = kwargs["t_training"] if "t_training" in kwargs else self.dataset["t_training"]
                except:
                    raise ValueError("Can't find (X,t,y) data to fit the model on")
                try:
                    ps_scores  = kwargs["ps_scores"] if "ps_scores" in kwargs else self.dataset["ps_scores"]
                except:
                    ps_scores = None
                try:
                    fun_to_prepare = bases['prepare_data'] if ["prepare_data"] in dir(
                        bases) else self.prepare_data
                    X_training, t_training, y_training, ps_scores = fun_to_prepare(X_training, t_training, y_training, ps_scores)
                except:
                    raise ValueError("Can't prepare data. Use either the default or implement new 'prepare_data' method")
                clsdict['fit'](self, X_training = X_training,
                               t_training = t_training,
                               y_training = y_training,
                               ps_scores = ps_scores)
            setattr(cls, 'fit', new_fit)

In [ ]:
#export 
class BaseITEEstimator(BaseEstimator, metaclass=FitTask):
    """Base class for estimating treatment effects
    
    This is a base class, that is also usable on its own to estimate treatment effects.
    It works with most models that have a .fit() method, but subclassing is also made
    straightforward throught a metaclass. """
    
    def __init__(self, 
                model: Union[str, Callable] = None, 
                dataset : Union[dict,None] = None,
                two_model: Union[bool,None] = None,
                ps_model : Union[Callable, None] = None,
                **kwargs)-> None:
        self.model = model
        self.dataset = dataset
        self.two_model = two_model
        self.ps_model = ps_model
        self.proba = False
        
    def _fit_ps_model(self, X_training, t_training):
        self.ps_model.fit(X_training, t_training)
        
    def prepare_data(self, X_training=None, t_training=None, y_training=None, ps_scores=None):
        if X_training is None:
            try:
                X_training = deepcopy(self.dataset["X_training"])
                y_training = deepcopy(self.dataset["y_training"])
                t_training = deepcopy(self.dataset["t_training"])
            except:
                raise ValueError("No data to fit the model on.")
        if np.unique(y_training).shape[0] == 2:
            self.proba = True
        if self.ps_model is not None or ps_scores is not None:
            if self.ps_model is not None:
                self._fit_ps_model(X_training, t_training)
                ps_scores = self.ps_model.predict(X_training)
            try:
                X_training = np.hstack((X_training, ps_scores[:,1].reshape((-1, 1))))
            except:
                raise ValueError(f"Shape of propensity scores is {ps_scores.shape},instead of (-1,1)")
        X_t_training = np.hstack((X_training, t_training.reshape(-1,1)))
        if self.two_model is False:
            X_training = deepcopy(X_t_training)

        return X_training, t_training, y_training, ps_scores
                
    def fit(self, 
            X_training, 
            t_training,
            y_training,
            ps_scores,
            **kwargs):
        "Fit function, using .fit() from other models"        
        if self.two_model is False or self.two_model is None:
            self.model.fit(X_training, y_training)
        else:
            control_ix = np.where(t_training == 0)[0]
            self.m1 = deepcopy(self.model)
            self.model.fit(X_training[control_ix,:], y_training[control_ix])
            self.m1.fit(X_training[-control_ix,:],   y_training[-control_ix])
                
    def _predict_bin_or_con(self, model, X):
        if self.proba:
            try:
                out = model.predict_proba(X)[:, 1]
            except:
                out = model.predict(X)
        else:
            out = model.predict(X)
        return out
                  
    def predict(self,
                    X: np.ndarray,
                    ps_scores = None,
                    **kwargs):
        if self.ps_model is not None or ps_scores is not None:
            if self.ps_model is not None:
                ps_scores = self.ps_model.predict_proba(X)
            X = np.hstack((X, ps_scores[:,1].reshape((-1, 1))))
        if self.two_model:
            m0_preds = self._predict_bin_or_con(self.model, X)
            m1_preds =  self._predict_bin_or_con(self.m1, X)
            ite = m1_preds - m0_preds
        else:
            try:
                X0 = np.hstack((X, np.zeros((X.shape[0], 1))))
                X1 = np.hstack((X, np.ones((X.shape[0], 1))))
                m1_preds = self._predict_bin_or_con(self.model, X1)
                m0_preds = self._predict_bin_or_con(self.model, X0)
                ite = m1_preds - m0_preds
            except:
                ite = self._predict_bin_or_con(self.model, X)
        return np.asarray(ite)

In [ ]:
#export
class BaseActiveLearner(BaseEstimator):
    """Basic of Active Learners later used, with the capability for treatment effects
    
    Inspired by modAL's BaseLearner, however modified for ITE estimation. Dataset is provided by 
    a dictionary for easier usage and less clutter.
    """
    def __init__(self,
                 estimator: BaseEstimator,
                 acquisition_function: Callable,
                 assignment_function: Union[Callable, list, None],
                 stopping_function: Union[Callable, list, None],
                 dataset: dict,
                 al_steps: int = 1,
                 offline = True,
                 **kwargs
                ) -> None:
        self.estimator = estimator
        if type(acquisition_function) is list:
            self.acquisition_function_list = acquisition_function
        else:
            self.acquisition_function = acquisition_function
            self.acquisition_function_list = []
        if type(assignment_function) is list:
            self.assignment_function_list = assignment_function
        else:
            self.assignment_function = assignment_function
            self.assignment_function_list = []
        self.stopping_function = stopping_function
        self.dataset = deepcopy(dataset)
        self.al_steps = al_steps
        self.current_step = 0
        self.next_query = True
        self.offline = offline
        if self.offline is False:
            self.dataset.__dict__["X_training"],\
            self.dataset.__dict__["t_training"],\
            self.dataset.__dict__["y_training"] = self.dataset.get_data(
                self.dataset.no_training)
            
        
    def _update_dataset(self, query_idx, **kwargs):
        """Moves data with query_idx indices from pool to training"""
        remove_data = kwargs["remove_data"] if "remove_data" in kwargs else True
        if type(self.dataset) is not dict:
            sds = self.dataset.__dict__
        else:
            sds = self.dataset
        for data in ["X", "t", "y"]:
            if sds[f"{data}_training"].shape[0] > 0 :
                try:
                    sds[f"{data}_training"] = np.concatenate((
                        sds[f"{data}_training"],
                        sds[f"{data}_pool"][query_idx,:]))
                except:
                    sds[f"{data}_training"] = np.concatenate((
                        sds[f"{data}_training"],
                        sds[f"{data}_pool"][query_idx]))
            else: 
                sds[f"{data}_training"] = sds[f"{data}_pool"][query_idx,:]
            if remove_data:
                sds[f"{data}_pool"] = np.delete(sds[f"{data}_pool"],
                                                             query_idx, 0)
        try:
            sds["y0_pool"] = np.delete(sds["y0_pool"],
                                                             query_idx, 0)
            sds["y1_pool"] = np.delete(sds["y1_pool"],
                                                             query_idx, 0)
        except:
            pass
                    
                
    def _select_counterfactuals(self, query_idx, treatment):
        "Rewrited observed outcome in pool for counterfactuals"
        self.dataset["y_pool"][query_idx] = np.where(treatment == 1,
                                                     self.dataset["y1_pool"][query_idx],
                                                     self.dataset["y0_pool"][query_idx])
        
        self.dataset["t_pool"][query_idx] = treatment
                
    def fit(self) -> None:
        self.estimator.fit(X_training = self.dataset["X_training"],
                           t_training = self.dataset["t_training"],
                           y_training = self.dataset["y_training"])
        return None
    
    def predict(self, X):
        return self.estimator.predict(X=X)
            
    def query(self, no_query = None, acquisition_function = None):
        """Main function to select datapoints"""
        if self.stopping_function is not None:
            self.next_query = self.stopping_function.check_rule(
                self.estimator, self.dataset, self.step)
        if self.next_query:
            if len(self.acquisition_function_list) == 0:
                if acquisition_function is None:
                    acquisition_function = self.acquisition_function
            else:
                if acquisition_function is None:
                    acquisition_function = self.acquisition_function
            if self.offline:
                X_new, query_idx = acquisition_function.select_data(self.estimator,
                                                              self.dataset, 
                                                              no_query)
            else:
                X_get = self.dataset.get_X(no_query = no_query)
                data_to_estimate = {"X_training":self.dataset["X_training"],
                                   "X_pool": X_get}
                decision_to_query = acquisition_function.select_data(self.estimator,
                                                              data_to_estimate, 
                                                              no_query,
                                                              offline = False)
                if decision_to_query:
                    self.X_to_add = X_get
                    X_new = X_get
                else:
                    X_new = None
                try:
                    query_idx = self.dataset.selected_ix
                except:
                    query_idx = None
        else:
            X_new, query_idx = None, None
        return X_new, query_idx
    
    def teach(self, query_idx=None, assignment_function = None, **kwargs):
        """Function to assign the selected labels """

        if len(self.assignment_function_list) == 0:
            if assignment_function is None:
                assignment_function = self.assignment_function
            else:
                raise ValueError("No assignment function provided")
        else:
            if assignment_function is None:
                assignment_function = self.assignment_function
        try:
            treatment = assignment_function.select_treatment(self.estimator,
                                                              self.dataset,
                                                              query_idx)
            if "y0_pool" in self.dataset:
                self._select_counterfactuals(query_idx, treatment)
            matching = treatment == self.dataset["t_pool"][query_idx]
            if matching.all():
                self._update_dataset(query_idx, **kwargs)
            elif matching.sum() > 0:
                self._update_dataset(query_idx[matching], **kwargs)
            else:
                pass
        except (AttributeError, IndexError) as e:
            treatment_to_add = self.dataset.get_t(X_new = self.X_to_add)
            y = self.dataset.get_y(X_new=self.X_to_add, t_new = treatment_to_add)
            self.dataset.__dict__["X_pool"] = self.X_to_add
            self.dataset.__dict__["t_pool"] = treatment_to_add
            self.dataset.__dict__["y_pool"] = y
            self._update_dataset(np.arange(y.shape[0]), remove_data=True)

        return None
    
    def score(self, metric="PEHE"):
        metrics = ["Qini", "PEHE", "Cgains", "decision"]
        if metric not in metrics:
            raise ValueError(f"Please use a valid error ({metrics}), {metric} is not valid")
        try:
            preds = self.estimator.predict(X=self.dataset["X_test"], return_mean=True)
        except:
            preds = self.estimator.predict(X=self.dataset["X_test"])
        if metric == "PEHE":
            try:
                sc = np.sqrt(np.mean(np.square(preds - self.dataset["ite_test"])))
            except KeyError:
                raise Error("Check if dataset contains true ITE values")
        elif metric == "decision":
            dec = np.where((preds >= 0) &( self.dataset["ite_test"] >= 0), 1, 0)
            sc = np.sum(dec)/self.dataset["ite_test"].shape[0]
        elif metric == "Qini":
            
        return sc
    
    def simulate(self, no_query: int = None, metric: str = "Qini") -> dict:
        ds = deepcopy(self.dataset)
        est = deepcopy(self.estimator)
        res = {}
        if len(self.acquisition_function_list) == 0:
            laf = [self.acquisition_function]
        else:
            laf = self.acquisition_function_list
        for af in laf:
            if no_query is None:
                no_query = af.no_query
            self.dataset = deepcopy(ds)
            self.estimator = deepcopy(est)
            self.current_step = 0
            res[af.name] = {}
            for i in range(1, self.al_steps+1):
                self.fit()
                X_new, query_idx = self.query(no_query=no_query, acquisition_function = af)
                if self.next_query:
                    if self.offline:
                        self.teach(query_idx)
                    else:
                        self.teach()
                    res[af.name][i] = self.score(metric=metric)
                    self.current_step += 1
        self.simulation_results = res
        return res
    
    def plot(self):
        pd.DataFrame(self.simulation_results).plot()

In [ ]:
#export
class BaseAcquisitionFunction():
    """Base class for acquisition functions"""
    def __init__(self,
                no_query: int = 1,
                method: str = "top",
                name: str = "base") -> None:
        self.no_query = no_query
        self.method = method
        self.name = name + "_" + str(no_query)
    
    def calculate_metrics(self, model, dataset) -> np.array:
        """Method to calculate base metrics for assignment function"""
        return np.arange(dataset["X_pool"].shape[0]) + 1
    
    def select_data(self, model, dataset, no_query, offline=True):
        """Based on the calculated metrics, select data and return X and indexes"""
        if no_query is None:
            no_query = self.no_query
        metrics = self.calculate_metrics(model, dataset)
        if offline:
            if self.method == "top":
                query_idx = np.argsort(np.asarray(metrics))[-no_query:][::-1]
                X_new = dataset["X_pool"][query_idx,:]
            else:
                raise NotImplementedError("Please use a method that is implemented")
            out = (X_new, query_idx)
        else:
            out = metrics.sum() >= metrics.shape[0]/2
        return out

In [ ]:
#export
class BaseAssignmentFunction():
    """Base class for assignment functions"""
    
    def __init__(self, base_selection = 0):
        self.base_selection = base_selection
    
    def select_treatment(self, model, dataset, query_idx):
        return dataset["t_pool"][query_idx]

In [ ]:
#export
class BaseStoppingRule():
    """Base class for providing a stopping rule for the active learner"""
    
    def __init__(self, budget=None):
        self.budget = budget
    
    def check_rule(self, model, dataset, step):
        if self.budget - step >= 0:
            out = False
        else:
            out = True
        return out

In [ ]:
#export
@dataclass
class BaseDataGenerator():
    ds: Union[dict, Callable]
    dgp_x : Union[Callable, None] = None
    dgp_t : Union[Callable, None] = None
    dgp_y : Union[Callable, None] = None
    no_training : int = 10
        
    def __getitem__(self, key):
        return super().__getattribute__(key)
        
    def get_X(self, method : str = "random", no_query : int = 1):
        if self.ds is not None:
            ix = np.random.randint(low = 0, high = self.ds["X_pool"].shape[0], size=(no_query))
            X_new = self.ds["X_pool"][ix,:]
            self.selected_ix = ix
        else:
            X_new = self.dgp_x(no_query)
        self.X_new = X_new
        return X_new
            
    def get_t(self, X_new=None):
        if X_new is None:
            X_new = self.X_new
        if self.ds is not None:
            t_new = self.ds["t_pool"][self.selected_ix]
        else:
            t_new = self.dgp_t(X_new)
        self.t_new = t_new
        return t_new
        
    def get_y(self, X_new=None, t_new=None):
        if X_new is None:
            X_new = self.X_new
        if t_new is None:
            t_new = self.t_new
        if self.ds is not None:
            y_new = np.where(t_new == 1,
                             self.ds["y1_pool"][self.selected_ix],
                             self.ds["y0_pool"][self.selected_ix])
        else:
            y_new = self.dgp_y(X_new, t_new)
        return y_new
            
    def get_data(self, no_query=1, as_test=False):
        X_new = self.get_X(no_query=no_query)
        t_new = self.get_t()
        y_new = self.get_y()
        if as_test:
            self.__dict__["X_test"] = X_new
        return (X_new, t_new, y_new)